# Decision Tree

### Goal: Build a Basic Decision Tree 

#### 1) Read in state.csv data
#### 2) Create basic daily volatility/momentum variables/signals
#### 3) Fit the tree 
#### 4) Predict returns

In [50]:
### Load Libraries
#Load the librarys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
from pandas_datareader import data as pdr
import io
import requests

# Technical Indicators
from technical_indicators import * # import all function


import plotly.tools as tls
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

init_notebook_mode(connected=True)


import warnings

## Read in Data - Online GitHub Repository

In [51]:
### Sam's Harddrive
# df_features = pd.read_csv("/Users/YoungFreeesh/Documents/GitHub/Stat479_Project/Data/prices.csv", sep= ',')
# df_features.head(6)

url = "https://raw.githubusercontent.com/SamIlic/Stat479_Project/master/Data/prices.csv"
df = pd.read_csv(url)

df.head(6)

,date,Ticker,open,close,low,high,volume
0,2010-01-04,AAL,4.840000,4.770000,4.660000,4.940000,9837300
1,2010-01-04,AAP,40.700001,40.380001,40.360001,41.040001,1701700
2,2010-01-04,AAPL,30.490000,30.572857,30.340000,30.642857,123432400
3,2010-01-04,ABC,26.290001,26.629999,26.139999,26.690001,2455900
4,2010-01-04,ABT,26.000339,26.129884,25.870792,26.177866,10829000
5,2010-01-04,ADBE,36.650002,37.090000,36.650002,37.299999,4710200


## Def'N Technical Indicators

#### Rate of Change n=2
- Daily Return

#### Standard Deviation
- Measure of Volatility

#### Relative Strength Index (RSI)
- Momentum Indicator (14 periods, days for us, is standard)
- RSI tracks the current strength/weakness of a stock based on the closing prices of a trading period
- Range = 0-100 --> oscillates between 0 & 100 (30 --> over-sold, 70 --> over-bought)

#### Average Directional Movement Index (ADX)
- Technical Analysis Metric
- Used to determine the relative Strength of a Trend, either Up/Down
- Measures the strength of the trend (regardless of direction) over time

#### Moving Average (MA)
- Average of that last n(21 or 14) periods

#### Exponential Moving Average (EMA)
- Weighted Moving Average
- Weights more recent price changes more heavily

#### Momentum
- Momentum is used as an oscillator to detect trends
- Think of momentum as a measure of continued performance (over/under)

#### Averate True Range (ATR)
- Technical Analysis Volatiltiy Indicator
- Measures volatiltiy by decomposing the entire range of an asset price for that period
- AVR is an average of the past 14 True Range (TR) indicators

#### Bollinger Band
- Creates 21 day 2SD Moving Average
- The closer a stock price moves to a band/2SD, the more overbought/underbought the stock

#### PPSR - Pivot Points via Supports and Resistances
- Pivot Point: Numerical Average of an assets High, Low, Closeing prices
- Helps predict short turn movements, can signam minor/major change in direction
- Calcualtes: 2 Support & 2 Resistance levels as well
- Used to choose entry and exit points for positions
- Good at identiying stop loss levels

#### Stochastic Oscillator
- Momentum Indicator
- Compares closing price of a security to the range of it's prices over a certain period
- The sensitivity of the oscillator to market movements is reducible by adjusting that time period or by taking

#### TRIX - Triple Exponential Average
- Momentum Indicator
- Shows the percentage change in a triple exponentially smoothed moving average
- When it is applied to triple smoothing of moving averages, it is designed to filter out price movements that are considered insignificant or unimportant. 
- Similar to MCAD

#### MACD - Moving Average Convergence Divergence
- Trend following momenum indicator
- Shows the relationsip between two exponential moving averages of a securities prices
- Results in MACD line, called signal line, can function as a trigger to buy/sell

#### Mass Index
- examines the range between high and low stock prices over a period of time.
- suggests that a reversal of the current trend will likely take place when the range widens beyond a certain point and then contracts
- Analyzes the narrowing and widening of trading ranges to identicy potential reversals based on market patterns that aren’t often considered by technical analysts largely focused on singular price and volume movements

#### Vortex Indicator VI
- A vortex indicator is used to spot trend reversals and confirm current trends

#### KST Oscillator - Know Sure Thing
- Momentum oscillator to make rate-of-change readings easier for traders to interpret.

#### True Strength Index (TSI)
- momentum indicator that helps traders identify short-term price swings while trading in the direction of the trend. 
- The indicator is useful for determining overbought and oversold conditions of a security by incorporating the short-term momentum of the market with the lagging benefits of moving averages.

#### Accumulation Distribution
- Accumulation/distribution is a momentum indicator that attempts to gauge supply and demand by determining whether investors are generally buying (accumulating) or selling (distributing) a certain stock. 
- The accumulation/distribution measure seeks to identify divergences between stock price and volume flow. 

#### Chaikin Oscillator
- measures the accumulation/distribution line of the moving average convergence divergence (MACD).
- This highlights the momentum implied by the accumulation/distribution line.

#### Money Flow Index (MFI)
- technical oscillator used for identifying overbought or oversold conditions for an individual security or an ETF or index.
- the money flow index incorporates both price and volume data, as opposed to just price. For this reason, some analysts know it as volume-weighted RSI
- ranges from 0-100

#### On-Balance Volume (OBV)
- momentum indicator that uses volume flow to predict changes in stock price

#### Force Index
- Force index is a numerical measure of the market power of a movement in the price of a stock. 

#### Ease of Movement
- The Ease of Movement technical indicator shows the relationship between price and volume. 
- Often times, traders use this indicator when assessing the strength of an underlying trend.

#### Commodity Channel Index (CCI) 
- momentum-based technical trading tool used most often to help determine when an investment vehicle is reaching a condition of being overbought or oversold.

#### Keltner Channel
- Keltner Channels are typically an alternative channel pattern used in conjunction with signals from standard envelope channels such as Bollinger Bands or Donchian Channels.

#### Ultimate Oscillator
- measure momentum across multiple timeframes
- The Ultimate Oscillator is a range-bound indicator with a value that fluctuates between 0 and 100. Similar to the Relative Strength Index (RSI), levels below 30 are deemed to be oversold, and levels above 70 are deemed to be overbought. 

#### Donchian Channel
- A Donchian Channel is an indicator formed by upper and lower bands around the price bars. The upper band marks the highest price of an issue for n periods while the lower band marks the lowest price for n periods. The area between the upper and lower bands represents the Donchian Channel. 






## Create Technical Indcators

In [53]:
# https://github.com/Crypto-toolbox/pandas-technical-indicators/blob/master/technical_indicators.py

### Get Data
###################################################################
###################################################################
###################################################################
"""
Get OHLCV data from Yahoo Finance
"""
def get_stock(stock,start,end):
    ### Get Data
    df = pdr.get_data_yahoo(stock, start, end)
    return df

# Get OHLCV FB stock data from Yahoo Finance
df = pd.DataFrame(get_stock('FB', '1/1/2016', '12/31/2016')).reset_index()



### Create Technical Indicators
###################################################################
###################################################################
###################################################################

df = rate_of_change(df, 2)
df = standard_deviation(df, 14)

df = relative_strength_index(df, 14) # periods
df = average_directional_movement_index(df, 14, 13) # n, n_ADX
df = moving_average(df, 21) # periods
df = exponential_moving_average(df, 21) # periods
df = momentum(df, 14) # 

df = average_true_range(df, 14)
df = bollinger_bands(df, 21)
df = ppsr(df)
df = stochastic_oscillator_k(df)
df = stochastic_oscillator_d(df, 14)
df = trix(df, 14)
df = macd(df, 26, 12)
df = mass_index(df)
df = vortex_indicator(df, 14)

df = kst_oscillator(df, 10, 10, 10, 15, 10, 15, 20, 30)
df = true_strength_index(df, 25, 13)

df = accumulation_distribution(df, 14)
df = chaikin_oscillator(df)
df = money_flow_index(df, 14)
df = on_balance_volume(df, 14)
df = force_index(df, 14)
df = ease_of_movement(df, 14)
df = commodity_channel_index(df, 14)
df = keltner_channel(df, 14)
df = ultimate_oscillator(df)
df = donchian_channel(df, 14)

# df.plot(y=['Close'])
# df.plot(y=['RSI_14'])
df.head(35)



,Date,High,Low,Open,Close,Volume,Adj Close,ROC_2,STD_14,RSI_14,...,MFI_14,OBV_14,Force_14,EoM_14,CCI_14,KelChM_14,KelChU_14,KelChD_14,Ultimate_Osc,Donchian_14
0,2016-01-04,102.239998,99.750000,101.949997,102.220001,37912400,102.220001,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-05,103.709999,101.669998,102.889999,102.730003,23258200,102.730003,0.004989,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-06,103.769997,100.900002,101.129997,102.970001,25096200,102.970001,0.002336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07,101.430000,97.300003,100.500000,97.919998,45172900,97.919998,-0.049043,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-08,100.500000,97.029999,99.879997,97.330002,35402300,97.330002,-0.006025,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-01-11,98.599998,95.389999,97.910004,97.510002,29932400,97.510002,0.001849,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2016-01-12,99.959999,97.550003,99.000000,99.370003,28395400,99.370003,0.019075,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2016-01-13,100.580002,95.209999,100.580002,95.440002,33410600,95.440002,-0.039549,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2016-01-14,98.870003,92.449997,95.849998,98.370003,48658600,98.370003,0.030700,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2016-01-15,96.379997,93.540001,93.980003,94.970001,45935600,94.970001,-0.034563,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
